In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pyspark.sql.functions as F

conf = SparkConf().setAppName("196").setMaster("local[4]")
spark = SparkSession.builder.config(conf = conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/31 01:34:48 WARN Utils: Your hostname, de24, resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp0s3)
25/07/31 01:34:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/31 01:34:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
'''
Table: Person

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| email       | varchar |
+-------------+---------+
id is the primary key (column with unique values) for this table.
Each row of this table contains an email. The emails will not contain uppercase letters.
 

Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.

For SQL users, please note that you are supposed to write a DELETE statement and not a SELECT one.

For Pandas users, please note that you are supposed to modify Person in place.

After running your script, the answer shown is the Person table. 
The driver will first compile and run your piece of code and then show the Person table. 
The final order of the Person table does not matter.

The result format is in the following example.

 

Example 1:

Input: 
Person table:
+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |
+----+------------------+
Output: 
+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
+----+------------------+
Explanation: john@example.com is repeated two times. We keep the row with the smallest Id = 1.
'''

In [3]:
data = [
(1,'john@example.com'),
(2,'bob@example.com' ),
(3,'john@example.com')    
]
schema = ['id','email']

In [18]:
df = spark.createDataFrame(data=data,schema=schema)
df.show()

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
|  3|john@example.com|
+---+----------------+



In PySpark DataFrames, we don’t have direct DELETE operations like traditional SQL. Instead, you filter out the rows you want to “delete” and overwrite or store the cleaned dataset. For your scenario, the logic is: keep only the rows whose id is the minimum for each email.


In [15]:
df_new = df.groupBy(F.col("email"))\
         .agg(   F.min(F.col("id")).alias("id"),
                 F.count(F.col("*")).alias("count")
             )\
         .select(F.col("id"),F.col("email"))
df_new.show()
df = df_new
df.show()

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+



## Anothor Method

In [21]:
#Step 1: Get min(id) per email
df_min_ids = df.groupBy(F.col("email")).agg(F.min(F.col("id")).alias("min_id"))

# Step 2: Join to original to retain full row
df_clean = df.alias("t1").join(df_min_ids.alias("t2"), 
                               (F.col("t1.email") == F.col("t2.email")) & 
                               (F.col("t1.id") == F.col("t2.min_id")), 
                               "inner") \
             .select(F.col("t1.*"))

df_clean.show()


+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+



## SQL Solution

<pre>

DELETE FROM Person 
    WHERE id NOT IN (
        SELECT x.id from (
            SELECT email, min(id) as id 
            FROM Person
            GROUP BY email
            HAVING count(*) >= 1    
        ) x
    );
</pre>